# Multitasking model implementation

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import os
os.chdir('/Users/arpitha/Documents/295B/musicautobot')

In [4]:
%pwd

'/Users/arpitha/Documents/295B/musicautobot'

In [5]:
from musicautobot.numpy_encode import *
from musicautobot.utils.file_processing import process_all, process_file
from musicautobot.config import *
from musicautobot.music_transformer import *
from musicautobot.multitask_transformer import *
from musicautobot.numpy_encode import stream2npenc_parts

In [6]:
from fastai.text import *

## MultitaskTransformer Training

### Pickling the dataset

In [23]:
# Location of your midi files
midi_path = Path('data/midi/examples')
midi_path.mkdir(parents=True, exist_ok=True)

# Location to save dataset
data_path = Path('data/numpy')
data_path.mkdir(parents=True, exist_ok=True)

data_save_name = 'musicitem_data_save.pkl'
s2s_data_save_name = 'multiitem_data_save.pkl'

### Gather midi dataset

In [24]:
midi_files = get_files(midi_path, '.mid', recurse=True); len(midi_files)

19

In [28]:
processors = [Midi2ItemProcessor()]
data = MusicDataBunch.from_files(midi_files, data_path, processors=processors, 
                                 encode_position=True, dl_tfms=mask_lm_tfm_pitchdur, 
                                 bptt=5, bs=2)
data.save(data_save_name)

/Users/arpitha/opt/anaconda3/lib/python3.8/site-packages/fastai/core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(a, dtype=dtype, **kwargs)


In [31]:
xb, yb = data.one_batch(); xb

{'msk': {'x': tensor([[  4, 145,   4, 145,  74],
          [  4, 145,   4, 145,   4]]),
  'pos': tensor([[8, 8, 8, 8, 8],
          [8, 8, 8, 8, 8]])},
 'lm': {'x': tensor([[139,  64, 145,  61, 145],
          [139,  64, 145,  61, 145]]),
  'pos': tensor([[8, 8, 8, 8, 8],
          [8, 8, 8, 8, 8]])}}

In [32]:
processors = [Midi2MultitrackProcessor()]
s2s_data = MusicDataBunch.from_files(midi_files, data_path, processors=processors, 
                                     preloader_cls=S2SPreloader, list_cls=S2SItemList,
                                     dl_tfms=melody_chord_tfm,
                                     bptt=5, bs=2)
s2s_data.save(s2s_data_save_name)

/Users/arpitha/Documents/295B/musicautobot/musicautobot/multitask_transformer/transform.py:38: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  def to_idx(self): return np.array((self.melody.to_idx(), self.chords.to_idx()))


In [34]:
xb, yb = s2s_data.one_batch(); xb

{'c2m': {'enc': tensor([[  5,   1,  61, 145,  59],
          [  5,   1,  69, 153,  66]]),
  'enc_pos': tensor([[0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0]]),
  'dec': tensor([[  6,   1,  85, 143,   8],
          [  6,   1,  78, 139,   8]]),
  'dec_pos': tensor([[0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0]])},
 'm2c': {'enc': tensor([[  6,   1,  85, 143,   8],
          [  6,   1,  78, 139,   8]]),
  'enc_pos': tensor([[0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0]]),
  'dec': tensor([[  5,   1,  61, 145,  59],
          [  5,   1,  69, 153,  66]]),
  'dec_pos': tensor([[0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0]])}}

### Loading the pre-trained model

In [38]:
# Data
data = MusicDataBunch.empty(data_path)
vocab = data.vocab

In [39]:
pretrained_path = data_path/'pretrained/MultitaskSmallKeyC.pth'

In [40]:
learn = multitask_model_learner(data, pretrained_path=pretrained_path)

### Choose existing midi file as a starting point

In [42]:
example_dir = midi_path
midi_files = get_files(example_dir, recurse=True, extensions='.mid'); midi_files[:5]

[PosixPath('data/midi/examples/Levels - Avicii - Verse.mid'),
 PosixPath('data/midi/examples/Scary Monsters And Nice Sprites - Skrillex - Pre-Chorus.mid'),
 PosixPath('data/midi/examples/Can You Feel The Love Tonight - Elton John - Verse.mid'),
 PosixPath('data/midi/examples/Locked Out Of Heaven - Bruno Mars - Chorus.mid'),
 PosixPath('data/midi/examples/In The Hall Of The Mountain King - Edvard Grieg - Intro.mid')]

In [43]:
midi_files = get_files(example_dir, '.mid', recurse=True); len(midi_files)

19

In [48]:
file = midi_files[0]; file

PosixPath('data/midi/examples/Levels - Avicii - Verse.mid')

In [49]:
# Encode file 
item = MusicItem.from_file(file, data.vocab)

x = item.to_tensor()
x_pos = item.get_pos_tensor()

In [50]:
# item.show()

In [51]:
item.play()

## Generate

MultitaskTransformer trains on 3 separate tasks. 
1. NextWord
2. Mask
3. Sequence to Sequence

Because we train on 3 separate tasks, we can actually generate some really cool note sequences.

1. NextWord/Autocomplete - Take a sequence of notes and predict the next note
 * 1a. Vanilla Language Model predictions - See [MusicTransformer](../music_transformer) project


2. Mask/Remix - Mask certain parts of song and remix those portions.
 * 2a. Note Masking - Mask all the note pitches and create a new sequence with different notes, but same exact rhythm
 * 2b. Duration Masking - Mask the note durations. Generate a new sequence with the same melody, but with a different rhythm


3. Seq2Seq/Translation - Generate melody from chords or vice versa. 
 * 3a. New Melody - Generate a new melody from existing chords
 * 3b. Harmonization - Generate chords to acompany an existing melody

## 1. NextWord/Autocomplete

Trim the song to only a few notes. Model will use these notes a seed and continue the idea

In [ ]:
seed_len = 6 # 4 beats = 1 bar
seed = item.trim_to_beat(seed_len)